- Reference
  - Blog: https://work-in-progress.hatenablog.com/entry/2019/04/06/113629
  - Source: https://github.com/eriklindernoren/Keras-GAN/blob/master/gan/gan.py
  - Source: https://github.com/eriklindernoren/Keras-GAN/pull/117
    - Added functionality to save/load Keras model for intermittent training.

In [1]:
import os

In [2]:
os.makedirs('data/images', exist_ok=True)
os.makedirs('data/saved_models', exist_ok=True)

In [3]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.models import Sequential, Model, model_from_json
from tensorflow.keras.optimizers import Adam

In [4]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime

In [25]:
class GAN():
    def __init__(self):
        self.history = pd.DataFrame({}, columns=['d_loss', 'acc', 'g_loss'])

        self.img_save_dir = 'data/images'
        self.model_save_dir = 'data/saved_models'
        self.discriminator_name = 'discriminator_model'
        self.generator_name = 'generator_model'
        self.combined_name = 'combined_model'

        self.discriminator = None
        self.generator = None
        self.combined = None

        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

    def init(self, loading=False):
        optimizer = Adam(0.0002, 0.5)

        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
                                   optimizer=optimizer,
                                   metrics=['accuracy'])
        if loading:
            self.load_model_weight(self.discriminator_name)

        self.generator = self.build_generator()
        if loading:
            self.load_model_weight(self.generator_name)

        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        validity = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, validity)
        self.combined.summary()
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)
        if loading:
            self.load_model_weight(self.combined_name)

    def build_generator(self):
        model = Sequential()

        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):
        model = Sequential()

        model.add(Flatten(input_shape=self.img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))

        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, sample_interval=-1, save_interval=-1):
        # Load the dataset
        (X_train, _), (_, _) = mnist.load_data()
        #print(X_train.shape)

        # Rescale -1 to 1
        X_train = X_train / 127.5 - 1.
        X_train = np.expand_dims(X_train, axis=3)
        #print(X_train.shape)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        print(datetime.datetime.now().isoformat(), 'Epoch Start')

        for epoch in range(epochs):
            # Select a random batch of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_imgs = self.generator.predict(noise)

            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            g_loss = self.combined.train_on_batch(noise, valid)

            self.history = self.history.append({'d_loss': d_loss[0], 'acc': d_loss[1], 'g_loss': g_loss}, ignore_index=True)

            if sample_interval > 0 and epoch % sample_interval == 0:
                print(datetime.datetime.now().isoformat(), '%d [D loss: %f, acc.: %.2f%%] [G loss: %f]' % (epoch, d_loss[0], 100*d_loss[1], g_loss))
                self.sample_images(epoch)

            if save_interval > 0 and epoch != 0 and epoch % save_interval == 0:
                self.save_model_weights_all()
                
        print(datetime.datetime.now().isoformat(), 'Epoch End')

    def sample_images(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0

        for i in range(r):
            for j in range(c):
                axs[i, j].imshow(gen_imgs[cnt, :, :, 0], cmap='gray')
                axs[i, j].axis('off')
                cnt += 1

        file_name = '{}.png'.format(epoch)
        file_path = os.path.join(self.img_save_dir, file_name)
        fig.savefig(file_path)

        plt.close()

    def plot_hisotry(self, columns=[]):
        if len(columns) == 0:
            columns = ['d_loss', 'acc', 'g_loss']
        self.history[columns].plot()
 
    def save_model_weights_all(self):
        self.save_model_weights(self.discriminator, self.discriminator_name)
        self.save_model_weights(self.generator, self.generator_name)
        self.save_model_weights(self.combined, self.combined_name)

    def save_model_weights(self, model, model_name):
        weights_path = os.path.join(self.model_save_dir, '{}.h5'.format(model_name))
        model.save_weights(weights_path)

        print('Weights saved.', model_name)

    def load_model_weight(self, model_name):
        model = None

        if model_name == self.discriminator_name:
            model = self.discriminator
        elif model_name == self.generator_name:
            model = self.generator
        elif model_name == self.combined_name:
            model = self.combined

        if not model:
            print('Model is not initialized.', model_name)
            return

        weights_path = os.path.join(self.model_save_dir, '{}.h5'.format(model_name))

        if not os.path.exists(weights_path):
            print('Not found h5 file.', model_name)
            return

        model.load_weights(weights_path)

        print('Weights loaded.', model_name)

In [26]:
gan = GAN()

In [27]:
#gan.init()
gan.init(loading=True)

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_6 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_42 (Dense)             (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_30 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
dense_43 (Dense)             (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_31 (LeakyReLU)   (None, 256)               0         
_________________________________________________________________
dense_44 (Dense)             (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
_______________________________________________

In [28]:
#gan.train(epochs=500, batch_size=32, sample_interval=50, save_interval=50)
gan.train(epochs=500, batch_size=32, sample_interval=50)
gan.save_model_weights_all()

2020-07-20T16:40:40.930278 Epoch Start
2020-07-20T16:40:41.984151 0 [D loss: 0.017323, acc.: 100.00%] [G loss: 4.603593]
2020-07-20T16:40:44.686288 50 [D loss: 0.043630, acc.: 100.00%] [G loss: 4.487054]
2020-07-20T16:40:47.771228 100 [D loss: 0.132579, acc.: 96.88%] [G loss: 3.762897]
2020-07-20T16:40:50.378977 150 [D loss: 0.180828, acc.: 95.31%] [G loss: 2.896070]
2020-07-20T16:40:52.967509 200 [D loss: 0.512427, acc.: 68.75%] [G loss: 2.078963]
2020-07-20T16:40:55.735869 250 [D loss: 0.722651, acc.: 42.19%] [G loss: 0.785459]
2020-07-20T16:40:58.321568 300 [D loss: 0.701244, acc.: 42.19%] [G loss: 0.665993]
2020-07-20T16:41:00.907402 350 [D loss: 0.661094, acc.: 50.00%] [G loss: 0.774907]
2020-07-20T16:41:03.797166 400 [D loss: 0.669651, acc.: 50.00%] [G loss: 0.668935]
2020-07-20T16:41:06.421565 450 [D loss: 0.671747, acc.: 43.75%] [G loss: 0.665527]
2020-07-20T16:41:08.977252 Epoch End
Weights saved. discriminator_model
Weights saved. generator_model
Weights saved. combined_model

In [29]:
gan.train(epochs=1000, batch_size=32, sample_interval=50)
gan.save_model_weights_all()

2020-07-20T16:41:43.543362 Epoch Start
2020-07-20T16:41:43.602838 0 [D loss: 0.646707, acc.: 60.94%] [G loss: 0.719015]
2020-07-20T16:41:46.254114 50 [D loss: 0.644686, acc.: 56.25%] [G loss: 0.700990]
2020-07-20T16:41:49.387355 100 [D loss: 0.638763, acc.: 60.94%] [G loss: 0.792076]
2020-07-20T16:41:52.140002 150 [D loss: 0.607266, acc.: 65.62%] [G loss: 0.807922]
2020-07-20T16:41:54.762389 200 [D loss: 0.630868, acc.: 59.38%] [G loss: 0.807604]
2020-07-20T16:41:57.533070 250 [D loss: 0.611223, acc.: 65.62%] [G loss: 0.824006]
2020-07-20T16:42:00.164798 300 [D loss: 0.626919, acc.: 67.19%] [G loss: 0.855750]
2020-07-20T16:42:02.789074 350 [D loss: 0.623483, acc.: 62.50%] [G loss: 0.778964]
2020-07-20T16:42:05.617582 400 [D loss: 0.619047, acc.: 68.75%] [G loss: 0.871488]
2020-07-20T16:42:08.233352 450 [D loss: 0.619326, acc.: 76.56%] [G loss: 0.822424]
2020-07-20T16:42:10.908828 500 [D loss: 0.624528, acc.: 70.31%] [G loss: 0.896437]
2020-07-20T16:42:13.529280 550 [D loss: 0.609691, a